In [2]:
# Import libraries
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from fredapi import Fred

In [3]:
api_key = 'your_api_key'

# Connect to FRED api
fred = Fred(api_key=api_key)
fred

In [19]:
def calculate_metrics(index_ticker, window, lookback_period=None):
  # Import reference interest rate
  rates = pd.DataFrame(fred.get_series('DGS10'))
  rates.columns = ['10-Year U.S. Treasury Yield (%)']

  # Import reference market index
  tickers = {'S&P500':'^GSPC',
             'S&P500 EW':'RSP',
            'Vanguard TMI':'VTI',
            'Russel 2000':'^RUT'}
  market_index = pd.DataFrame(yf.download(tickers[index_ticker], start='1960-01-01', end='2023-12-31')['Adj Close'])
  market_index.columns = [index_ticker]

  # Combine dataframes
  rates_vs_market = rates.merge(market_index, how='left', left_index=True, right_index=True)
  
  # Adjust data to lookback period
  if lookback_period is not None:
    rates_vs_market = rates_vs_market[rates_vs_market.index.year >= (datetime.date.today().year - lookback_period)]

  # Resample data
  rates_vs_market = rates_vs_market.resample('M').last()

  # Calculate rates and market changes
  rates_vs_market['Yield Change'] = 100*(rates_vs_market['10-Year U.S. Treasury Yield (%)'] - rates_vs_market['10-Year U.S. Treasury Yield (%)'].shift(1))
  rates_vs_market['Window Yield Change'] = rates_vs_market['Yield Change'].rolling(window).sum()
  rates_vs_market['Window Market Returns'] = 100*rates_vs_market[index_ticker].pct_change(window)
  rates_vs_market['Following Window Market Returns'] = rates_vs_market['Window Market Returns'].shift(-window)
  rates_vs_market.dropna(inplace=True)

  return rates, market_index, rates_vs_market

In [20]:
# Run Function
index_ticker = 'S&P500'
window = 1
lookback_period = None

rates, market_index, rates_vs_market = calculate_metrics(index_ticker, window, lookback_period)
rates_vs_market

[*********************100%%**********************]  1 of 1 completed


,10-Year U.S. Treasury Yield (%),S&P500,Yield Change,Window Yield Change,Window Market Returns,Following Window Market Returns
1962-02-28,4.00,69.959999,-10.0,-10.0,1.626965,-0.586044
1962-03-31,3.86,69.550003,-14.0,-14.0,-0.586044,-6.196988
1962-04-30,3.86,65.239998,0.0,0.0,-6.196988,-8.599014
1962-05-31,3.90,59.630001,4.0,4.0,-8.599014,-8.183802
1962-06-30,4.00,54.750000,10.0,10.0,-8.183802,6.356164
...,...,...,...,...,...,...
2023-07-31,3.97,4588.959961,16.0,16.0,3.113893,-1.771639
2023-08-31,4.09,4507.660156,12.0,12.0,-1.771639,-4.871937
2023-09-30,4.59,4288.049805,50.0,50.0,-4.871937,-2.197969
2023-10-31,4.88,4193.799805,29.0,29.0,-2.197969,8.917927


In [21]:
# Plot rates behavior
fig = px.line(rates,
           x=rates.index,
           y=['10-Year U.S. Treasury Yield (%)'],
                 width=900)

fig.update_layout(title='10-Year U.S. Treasury Yield (%)',
    xaxis_title="Date",
    yaxis_title="Yield (%)",
    legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [22]:
# Plot rates change
fig = px.line(rates_vs_market,
           x=rates_vs_market.index,
           y=['Yield Change'],
                 width=900)

fig.update_layout(title='10-Year U.S. Treasury Yield Change (bps)',
    xaxis_title="Date",
    yaxis_title="Monthly Yield Change (bps)",
    legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [23]:
# Plot reference index
fig = px.line(market_index,
           x=market_index.index,
           y=index_ticker,
                 width=900)

fig.update_layout(title=f"{index_ticker} Index",
    xaxis_title="Date",
    yaxis_title="Index",
    legend_title="Legend")

fig.show()

In [24]:
# Plot bps changes in histogram
fig = px.histogram(rates_vs_market,
                   x=['Yield Change'],
                   marginal='box',
                   width=900)

fig.update_layout(title='10-Year U.S. Treasury Yield bps change',
    legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [25]:
# Fit OLS regression model
model = sm.OLS(rates_vs_market['Window Market Returns'], sm.add_constant(rates_vs_market['Window Yield Change'])).fit()
alpha = model.params.iloc[0]
beta = model.params.iloc[1]
rates_vs_market['Regression'] = alpha + (beta*rates_vs_market['Window Market Returns'])

# Plot rates vs market
fig = px.scatter(rates_vs_market,
           x='Window Yield Change',
           y=['Window Market Returns', 'Regression'],
                 width=900)

fig.update_layout(title="Yield Changes vs Market Returns",
    xaxis_title="Yield Change (bps)",
    yaxis_title="Market Returns (%)",
    legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [26]:
# Fit OLS regression model
model = sm.OLS(rates_vs_market['Following Window Market Returns'], sm.add_constant(rates_vs_market['Window Yield Change'])).fit()
alpha = model.params.iloc[0]
beta = model.params.iloc[1]
rates_vs_market['Regression'] = alpha + (beta*rates_vs_market['Following Window Market Returns'])

# Plot rates vs market in shifted window
fig = px.scatter(rates_vs_market,
           x='Window Yield Change',
           y=['Following Window Market Returns', 'Regression'],
                 width=900)

fig.update_layout(title="Yield Changes vs Market Returns",
    xaxis_title="Yield Change (bps)",
    yaxis_title="Market Returns (%)",
    legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [14]:
# Create subplots with different windows
window_list = [3, 6, 12, 24]
lookback_period = None
results = {}
fig = make_subplots(rows=2, cols=2)

for window in window_list:

  # Calculate metrics
  rates, market_index, rates_vs_market = calculate_metrics(index_ticker, window, lookback_period)

  # Fit regression model
  #model = sm.OLS(rates_vs_market['Window Market Returns'], sm.add_constant(rates_vs_market['Window Yield Change'])).fit()
  #alpha = model.params[0]
  #beta = model.params[1]
  #rates_vs_market['Regression'] = alpha + (beta*rates_vs_market['Window Market Returns'])
  
  # Create chart
  trace = go.Scatter(
      x=rates_vs_market['Window Yield Change'],
      y=rates_vs_market['Window Market Returns'],
      mode='markers',
      marker=dict(size=4))
  results[window] = trace

# Create subplots
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("3-Month Window", "6-Month Window", "12-Month Window", "24-Month Window"))

fig.add_trace(results[3], row=1, col=1)
fig.add_trace(results[6], row=1, col=2)
fig.add_trace(results[12], row=2, col=1)
fig.add_trace(results[24], row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Yield Change (bps)", row=1, col=1)
fig.update_xaxes(title_text="Yield Change (bps)", row=1, col=2)
fig.update_xaxes(title_text="Yield Change (bps)", row=2, col=1)
fig.update_xaxes(title_text="Yield Change (bps)", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Market Returns (%)", row=1, col=1)
fig.update_yaxes(title_text="Market Returns (%)", row=1, col=2)
fig.update_yaxes(title_text="Market Returns (%)", row=2, col=1)
fig.update_yaxes(title_text="Market Returns (%)", row=2, col=2)

fig.update_layout(height=800, width=950, title_text="Yield Changes vs Market Returns", showlegend=False)
fig.show()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [15]:
# Create subplots with different windows
window_list = [3, 6, 12, 24]
results = {}
fig = make_subplots(rows=2, cols=2)

for window in window_list:

  # Calculate metrics
  rates, market_index, rates_vs_market = calculate_metrics(index_ticker, window, lookback_period)

  # Fit regression model
  #model = sm.OLS(rates_vs_market['Following Window Market Returns'], sm.add_constant(rates_vs_market['Window Yield Change'])).fit()
  #alpha = model.params[0]
  #beta = model.params[1]
  #rates_vs_market['Regression'] = alpha + (beta*rates_vs_market['Following Window Market Returns'])

  # Create chart
  trace = go.Scatter(
      x=rates_vs_market['Window Yield Change'],
      y=rates_vs_market['Following Window Market Returns'],
      mode='markers',
      marker=dict(size=4))
  results[window] = trace

# Create subplots
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("3-Month Window", "6-Month Window", "12-Month Window", "24-Month Window"))

fig.add_trace(results[3], row=1, col=1)
fig.add_trace(results[6], row=1, col=2)
fig.add_trace(results[12], row=2, col=1)
fig.add_trace(results[24], row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Yield Change (bps)", row=1, col=1)
fig.update_xaxes(title_text="Yield Change (bps)", row=1, col=2)
fig.update_xaxes(title_text="Yield Change (bps)", row=2, col=1)
fig.update_xaxes(title_text="Yield Change (bps)", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Market Returns (%)", row=1, col=1)
fig.update_yaxes(title_text="Market Returns (%)", row=1, col=2)
fig.update_yaxes(title_text="Market Returns (%)", row=2, col=1)
fig.update_yaxes(title_text="Market Returns (%)", row=2, col=2)

fig.update_layout(height=800, width=950, title_text="Yield Changes vs Following Window Market Returns", showlegend=False)
fig.show()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [16]:
# Import FED Funds Rate
fed_funds_rate = pd.DataFrame(fred.get_series('EFFR'))
fed_funds_rate.columns = ['FED Funds Rate (%)']

# Import 10 year interest rates metrics
ten_year_treasury_yield = pd.DataFrame(fred.get_series('DGS10'))
ten_year_treasury_yield.columns = ['10-Year U.S. Treasury Yield (%)']

# Import market index
market_index = pd.DataFrame(yf.download('^GSPC', start='1960-01-01', end='2023-12-31')['Adj Close'])
market_index.columns = ['S&P500']

# Combine dataframes
fed_rates_vs_market = fed_funds_rate.merge(market_index, how='inner', left_index=True, right_index=True)
fed_rates_vs_market = fed_rates_vs_market.merge(ten_year_treasury_yield, how='inner', left_index=True, right_index=True)
fed_rates_vs_market = fed_rates_vs_market[fed_rates_vs_market.index.year > 2021]
fed_rates_vs_market['S&P500'] = fed_rates_vs_market['S&P500']/fed_rates_vs_market['S&P500'].iloc[0]

# Plot data
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=fed_rates_vs_market.index, y=fed_rates_vs_market['FED Funds Rate (%)'], name='FED Funds Rate (%)'), secondary_y=False)
fig.add_trace(go.Scatter(x=fed_rates_vs_market.index, y=fed_rates_vs_market['S&P500'], name='S&P 500'), secondary_y=True)
fig.add_trace(go.Scatter(x=fed_rates_vs_market.index, y=fed_rates_vs_market['10-Year U.S. Treasury Yield (%)'], name='10-Year U.S. Treasury Yield (%)'), secondary_y=False)
fig.update_yaxes(title_text="S&P 500 Compounded Normalized Returns", secondary_y=True)
fig.update_layout(title='FED Funds Rates vs Market Returns',
                    xaxis_title='Date',
                    yaxis_title='Rate (%)',
                height=600,
                width=950,
                legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

[*********************100%%**********************]  1 of 1 completed


In [17]:
fed_rates_vs_market.corr()

,FED Funds Rate (%),S&P500,10-Year U.S. Treasury Yield (%)
FED Funds Rate (%),1.000000,0.010317,0.872457
S&P500,0.010317,1.000000,-0.133194
10-Year U.S. Treasury Yield (%),0.872457,-0.133194,1.000000


In [18]:
# Fit OLS regression model
fed_rates_vs_market.dropna(inplace=True)
model = sm.OLS(fed_rates_vs_market['S&P500'], sm.add_constant(fed_rates_vs_market['10-Year U.S. Treasury Yield (%)'])).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 S&P500   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     8.687
Date:                Mon, 11 Dec 2023   Prob (F-statistic):            0.00336
Time:                        17:05:13   Log-Likelihood:                 717.12
No. Observations:                 483   AIC:                            -1430.
Df Residuals:                     481   BIC:                            -1422.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.9030      0.011     80.010      0.000       0.881       0.925
10-Year U.S. Treasury Yield (%)    -0.0094      0.003     -2.947      0.003      -0.016      -0.003
==============================================================================
Omnibus:                      114.125   Durbin-Watson:                   0.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.558
Skew:                          -0.024   Prob(JB):                     2.08e-05
Kurtosis:                       1.966   Cond. No.                         17.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""